In [51]:
 
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import load_model


import numpy as np
from pdf2image import convert_from_path

In [52]:
def load_pdf_to_analyze(path):
    image = convert_from_path(path, dpi=75)
    image = np.array(image)
    #image = format_image(image)
    # Convert RGB to grayscale (monochrome)
    if image.shape[-1] == 3:  # Ensure it's an RGB image
        image = tf.image.rgb_to_grayscale(image)
    
    image = tf.convert_to_tensor(image, dtype=tf.uint8) 
    
    # Set the target height and width (adjust as needed)
    target_size = (1241, 1000)  # Example target width and height

    # Resize all images
    image_resized = tf.image.resize(image, target_size)
    result = tf.data.Dataset.from_tensor_slices({ "image": image_resized})
    return result

def format_image(data):        
    image = data["image"]
    image = tf.reshape(image, [-1])
    image = tf.cast(image, 'float32')
    image = image / 255.0
    return image

In [53]:
class_codes = {"ECG_GCG" : 0, "ECG_FCG" : 1, "ECG_GCG_EC" : 2, "GCG_GCG_GCG" : 3}

In [54]:

model = load_model("model.keras")


In [61]:
path = 'C:/Users/jonhe/OneDrive/Documents/Hackathons/AEC Construction 2025/Data/ECG_FCG/DF2.10.pdf'

input = load_pdf_to_analyze(path)

input_formatted = input.map(format_image)

In [62]:
input_formatted

<_MapDataset element_spec=TensorSpec(shape=(1241000,), dtype=tf.float32, name=None)>

In [63]:
# batch_size = 32

# input_formatted = input_formatted.batch(batch_size)  # Ensure batching

In [64]:
# for batch in input_formatted.take(1):
#     print(batch.shape)

In [65]:
print(model.input_shape)


(None, 1241000)


In [66]:
for batch in input_formatted.take(1):
    batch = tf.expand_dims(batch, axis=0)  # Convert shape (1241000,) → (1, 1241000)
    model(batch)  # Now it matches (None, 1241000)

In [ ]:
for batch in input_formatted.take(1):  # Take a single batch
    batch = tf.expand_dims(batch, axis=0)  # Add batch dimension (1, 1241000)
    #print(batch.shape)  # Should be (1, 1241000)
    outputs=model(batch)  # Pass the tensor batch to the model

    print("Model Outputs:", outputs.numpy())  # Convert tensor to NumPy for readability

(1, 1241000)
Model Outputs: [[0.012424   0.0205664  0.92927426 0.03773531]]


In [88]:
item_nr = outputs.numpy()[0].argmax()

print('The category is ' + list(class_codes.keys())[item_nr] +'.')

The category is ECG_GCG_EC.
